In [47]:
import numpy as np
rng = np.random.default_rng()
N = 100
path_hamiltonian = np.zeros((N, N))

#creating a path hamiltonian and random hamiltonian for testing
for i in range(N):
    for j in range(N):
        if (i+1)%N == j or (i-1)%N == j:
            path_hamiltonian[i][j] = -1


A = np.zeros((N, N))
A.fill(.5)
random_hamiltonian = 2*(rng.binomial(1, .5, (N, N)) - A)
random_hamiltonian = (random_hamiltonian + random_hamiltonian.T)/2

for i in range(N):
    random_hamiltonian[i][i] = 0
    
#produces a hamiltonian from sampler data file
def hamiltonian(filename):
    file = pd.read_csv(filename)
    size = len(file.columns)-1
    H = np.zeros((size, size))
    
    for i in range(size):
            for j in range(size-1):
                H[i][j] = file.iat[j, i+1]
            
    return H

#computes the number of indices where two states differ
def discrepancy(state1, state2):
    d1 = sum(abs(state1+state2))/2
    d2 = sum(abs(state1-state2))/2
    return min(d1, d2)

#finds the smallest discrepency between states of minimal energy
def min_discrepancy(min_state, filename):
    row = 0
    sampler_data = pd.read_csv(filename)
    
    sampler_min_energy = sampler_data.at[row, 'energy']
    min_discr = discrepancy(min_state, [sampler_data.iat[row, i+1] for i in range(len(sampler_data.columns)-4)])
    
    while sampler_data.at[row+1, 'energy'] == sampler_min_energy:
        row += 1
        row_discrepancy = discrepancy(min_state, [sampler_data.iat[row, i+1] for i in range(len(sampler_data.columns)-4)])
        if row_discrepancy < min_discr:
            min_discr = row_discrepancy
        
    return min_discr  

In [49]:
import pandas as pd
import time
import ParallelTempering

#time_data = pd.read_csv("Data/Time_Data.csv")
comparison_data = {}
comparison_data_rows = ['PT Min energy', 'DWave Min Energy', 'Discrepancy']
start_time = time.time()

for i in range(1, 11):
    #cliqfile = "Data/Clique_Hamiltonian_" + str(i)
    #pegfile = "Data/Pegasus_Hamiltonian_" + str(i)
    gaussfile = "Data/Gauss_Hamiltonian_" + str(i)
    
    #cliq_hamiltonian = hamiltonian(cliqfile)
    #peg_hamiltonian = hamiltonian(pegfile)
    gauss_hamiltonian = hamiltonian(gaussfile)
    
    temperatures = np.arange(1, 10, .5)
    
    cliq_time = 0
    peg_time = 0
    
    if i < 6:
        #cliq_time = 3
        #peg_time = 2
        gauss_time = 2
    else:
        #cliq_time = 5
        #peg_time = 60
        gauss_time = 40
    
    #cliq_results = ParallelTempering.timed_find_min(cliq_hamiltonian, temperatures, cliq_time)
    #print("Finished Cliq " + str(i) + " at " + str(time.time() - start_time))
    #peg_results = ParallelTempering.timed_find_min(peg_hamiltonian, temperatures, peg_time)
    #print("Finished Peg " + str(i) + " at " + str(time.time() - start_time))
    gauss_results = ParallelTempering.timed_find_min(gauss_hamiltonian, temperatures, gauss_time)
    print("Finished Gauss " +str(i) + " at " + str(time.time()-start_time))
    
    #dwave_cliq = pd.read_csv("Data/Clique_Sampler_Data_" + str(i))
    #dwave_peg = pd.read_csv("Data/Pegasus_Sampler_Data_" + str(i))
    dwave_gauss = pd.read_csv("Data/Gauss_Sampler_Data_"+str(i))
    
    #comparison_data["Clique "+ str(i)] = [cliq_results[1], 2*dwave_cliq.at[0, 'energy'], min_discrepancy(cliq_results[0], "Data/Clique_Sampler_Data_" + str(i))]
    #comparison_data["Pegasus "+ str(i)] = [peg_results[1], 2*dwave_peg.at[0, 'energy'], min_discrepancy(peg_results[0], "Data/Pegasus_Sampler_Data_" + str(i))]
    comparison_data["Gauss "+ str(i)] = [gauss_results[1], 2*dwave_gauss.at[0, 'energy'], min_discrepancy(gauss_results[0], "Data/Gauss_Sampler_Data_"+str(i))]
    

Finished Gauss 1 at 2.0900118350982666
Finished Gauss 2 at 4.166111946105957
Finished Gauss 3 at 6.242712020874023
Finished Gauss 4 at 8.311439752578735
Finished Gauss 5 at 10.380598068237305
Finished Gauss 6 at 56.60065984725952
Finished Gauss 7 at 102.23094081878662
Finished Gauss 8 at 147.98659086227417
Finished Gauss 9 at 193.59726905822754
Finished Gauss 10 at 239.3174500465393


In [21]:
#data = pd.DataFrame(comparison_data, index = comparison_data_rows)
#data.to_csv("Data/Comparison_data.csv")